In [ ]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
import geopandas as gpd

def bulk_climate_vulnerability_temperature_maize(temperature_data, temperature_variable):
    # Define the threshold temperature values
    temperature_thresholds = {
        'low': 16.0,   # Below this temperature, maize yield is significantly reduced
        'medium': 25.0,  # Temperature range where maize yield is relatively stable
        'high': 30.0,  # Above this temperature, maize yield is significantly reduced
    }
    temperature = temperature_data[temperature_variable]

    #Vulnerability thresholds
    if temperature < temperature_thresholds['low']:
        vulnerability_score = 1.0
    elif temperature >= temperature_thresholds['low'] and temperature < temperature_thresholds['medium']:
        vulnerability_score = (temperature_thresholds['medium'] - temperature) / (temperature_thresholds['medium'] - temperature_thresholds['low'])
    elif temperature >= temperature_thresholds['medium'] and temperature <= temperature_thresholds['high']:
        vulnerability_score = (temperature - temperature_thresholds['medium']) / (temperature_thresholds['high'] - temperature_thresholds['medium'])
    else:
        vulnerability_score = 1.0
    return vulnerability_score

#climate data!
climate_data = xr.open_dataset('climate_data.nc')
temperature_variable = 'temperature'  # Replace with the actual variable name in the dataset
temperature = climate_data[temperature_variable]
output_geojson = 'vulnerability_score.geojson'

#Vulnerability raster
height, width = temperature.shape
transform = from_origin(climate_data.longitude[0], climate_data.latitude[-1], climate_data.longitude[1]-climate_data.longitude[0], climate_data.latitude[0]-climate_data.latitude[1])
vulnerability_score_raster = rasterize([(i, j) for i in range(width) for j in range(height)], transform=transform, out_shape=(height, width), fill=0, all_touched=False, dtype='float32', default_value=0)
for i in range(width):
    for j in range(height):
        temperature_value = temperature[j, i].item()
        vulnerability_score = bulk_climate_vulnerability_temperature_maize(temperature_data=temperature_value, temperature_variable=temperature_variable)
        vulnerability_score_raster[j, i] = vulnerability_score
vulnerability_score_df = gpd.GeoDataFrame({'vulnerability_score': vulnerability_score_raster.flatten()}, geometry=gpd.points_from_xy(climate_data.longitude.values, climate_data.latitude.values))
vulnerability_score_df.to_file(output_geojson, driver='GeoJSON')
print("Vulnerability score exported as GeoJSON: {}".format(output_geojson))

#colour coding of raster can then be set in GIS
